# All Indicator Signals Added to Database

In [1]:
import pandas as pd
import numpy as np
import sqlite3
from apscheduler.schedulers.blocking import BlockingScheduler


In [2]:

df = pd.read_csv('ohlc_1m_data_last_day.csv', parse_dates=['timestamp'])
df.set_index('timestamp', inplace=True)

In [3]:


def resample_signals(df):
    # Super Trend
    df['ATR'] = df['high'] - df['low']
    df['ATR'] = df['ATR'].rolling(window=14).mean()
    multiplier = 3
    df['basic_upper_band'] = (df['high'] + df['low']) / 2 + multiplier * df['ATR']
    df['basic_lower_band'] = (df['high'] + df['low']) / 2 - multiplier * df['ATR']
    df['Super_Trend'] = df['basic_upper_band'].where(df['close'] > df['basic_upper_band'].shift(1),
                                                     df['basic_lower_band'].where(df['close'] < df['basic_lower_band'].shift(1)))
    df['Super_Trend'].ffill(inplace=True)
    df['Super_Trend_signal'] = np.where(df['close'] > df['Super_Trend'], 1, -1)

    # EMA
    df['EMA'] = df['close'].ewm(span=20, adjust=False).mean()
    df['EMA_signal'] = np.where(df['close'] > df['EMA'], 1, -1)

    # Hybrid Indicator
    donchian_period = 20
    df['Donchian_High'] = df['high'].rolling(window=donchian_period).max()
    df['Donchian_Low'] = df['low'].rolling(window=donchian_period).min()
    df['cumulative_volume'] = df['volume'].cumsum()
    df['cumulative_price_volume'] = (df['close'] * df['volume']).cumsum()
    df['VWAP'] = df['cumulative_price_volume'] / df['cumulative_volume']
    moving_average_period = 50
    df['Moving_Avg'] = df['close'].rolling(window=moving_average_period).mean()
    df['Breadth_Indicator'] = np.where(df['close'] > df['Moving_Avg'], 1, 0)
    
    df['Hybrid_signal'] = np.where(
        (df['close'] > df['Donchian_High'].shift(1)) & (df['close'] > df['VWAP']) & (df['Breadth_Indicator'] == 1), 1,
        np.where(
            (df['close'] < df['Donchian_Low'].shift(1)) & (df['close'] < df['VWAP']) & (df['Breadth_Indicator'] == 0), -1, np.nan
        )
    )
    df['Hybrid_signal'].fillna(-1, inplace=True)
    df['Hybrid_signal'] = df['Hybrid_signal'].astype(int)

    # Stochastic Oscillator
    n = 14
    df['Lowest_Low'] = df['low'].rolling(window=n).min()
    df['Highest_High'] = df['high'].rolling(window=n).max()
    df['%K'] = ((df['close'] - df['Lowest_Low']) / (df['Highest_High'] - df['Lowest_Low'])) * 100
    df['%D'] = df['%K'].rolling(window=3).mean()

    df['Stochastic_signal'] = np.where(
        (df['%K'] > df['%D']) & (df['%K'].shift(1) <= df['%D'].shift(1)), 1,
        np.where(
            (df['%K'] < df['%D']) & (df['%K'].shift(1) >= df['%D'].shift(1)), -1, np.nan
        )
    )
    df['Stochastic_signal'].fillna(-1, inplace=True)
    df['Stochastic_signal'] = df['Stochastic_signal'].astype(int)

    # Bollinger Bands
    window = 20
    no_of_std = 2
    df['MA'] = df['close'].rolling(window=window).mean()
    df['BB_Upper'] = df['MA'] + (df['close'].rolling(window=window).std() * no_of_std)
    df['BB_Lower'] = df['MA'] - (df['close'].rolling(window=window).std() * no_of_std)
    df['BB_signal'] = np.where(df['close'] < df['BB_Lower'], 1, np.where(df['close'] > df['BB_Upper'], -1, np.nan))
    df['BB_signal'].fillna(-1, inplace=True)
    df['BB_signal'] = df['BB_signal'].astype(int)
    resampled_df = df.resample('4H').agg({
        'Super_Trend_signal': 'last',
        'EMA_signal': 'last',
        'Hybrid_signal': 'last',
        'Stochastic_signal': 'last',
        'BB_signal': 'last'
    }).dropna()

    return resampled_df
resampled_df = resample_signals(df)
print(resampled_df)


C:\Users\MTariq\AppData\Local\Temp\ipykernel_12344\550431303.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Super_Trend'].ffill(inplace=True)
C:\Users\MTariq\AppData\Local\Temp\ipykernel_12344\550431303.py:41: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, whe

                     Super_Trend_signal  EMA_signal  Hybrid_signal  \
timestamp                                                            
2020-01-01 00:00:00                  -1          -1             -1   
2020-01-01 04:00:00                  -1          -1             -1   
2020-01-01 08:00:00                   1          -1             -1   
2020-01-01 12:00:00                  -1           1             -1   
2020-01-01 16:00:00                  -1          -1             -1   
...                                 ...         ...            ...   
2024-08-11 08:00:00                   1           1             -1   
2024-08-11 12:00:00                  -1          -1             -1   
2024-08-11 16:00:00                   1          -1             -1   
2024-08-11 20:00:00                  -1          -1             -1   
2024-08-12 00:00:00                  -1          -1             -1   

                     Stochastic_signal  BB_signal  
timestamp                            

#  Inserting SIGNALS

In [4]:
import pandas as pd
import sqlite3
def convert_timestamps(df):
    df.reset_index(inplace=True)
    df['timestamp'] = df['timestamp'].astype(str)
    return df
conn = sqlite3.connect('indicators.db')
cursor = conn.cursor()
cursor.execute('''
CREATE TABLE IF NOT EXISTS Indicators (
    date_time TEXT PRIMARY KEY,
    Super_Trend_signal INTEGER,
    EMA_signal INTEGER,
    Hybrid_signal INTEGER,
    Stochastic_signal INTEGER,
    BB_signal INTEGER
)
''')
def insert_data(df):
    df = convert_timestamps(df) 
    data = df.values.tolist()
    cursor.executemany('''
    INSERT OR REPLACE INTO Indicators (date_time, Super_Trend_signal, EMA_signal, Hybrid_signal, Stochastic_signal, BB_signal)
    VALUES (?, ?, ?, ?, ?, ?)
    ''', data)
    conn.commit()
insert_data(resampled_df)
conn.close()


# DATABASE APPEND SIGNALS with LSTM 


In [8]:
import pandas as pd
import sqlite3
lstm_df = pd.read_csv('LSTM_signals.csv')
conn = sqlite3.connect('indicators.db')
cursor = conn.cursor()
existing_table_name = 'indicators'
cursor.execute(f"SELECT name FROM sqlite_master WHERE type='table' AND name='{existing_table_name}';")
table_exists = cursor.fetchone()
if table_exists:
    cursor.execute(f"PRAGMA table_info({existing_table_name});")
    columns = [column[1] for column in cursor.fetchall()] 
    if 'LSTM_Signals' not in columns:
        cursor.execute(f"ALTER TABLE {existing_table_name} ADD COLUMN LSTM_Signals INTEGER;")
        conn.commit()
existing_df = pd.read_sql(f'SELECT * FROM {existing_table_name}', conn)
if len(lstm_df) > len(existing_df):
    lstm_df = lstm_df.head(len(existing_df))
elif len(lstm_df) < len(existing_df):
    padding_length = len(existing_df) - len(lstm_df)
    padding_df = pd.DataFrame({'LSTM_Signals': [0] * padding_length})
    lstm_df = pd.concat([lstm_df, padding_df], ignore_index=True)
existing_df['index'] = existing_df.index
lstm_df['index'] = lstm_df.index
merged_df = pd.merge(existing_df, lstm_df[['index', 'LSTM_Signals']], on='index', how='left')
merged_df.drop(columns='index', inplace=True)
cursor.execute(f"DROP TABLE IF EXISTS {existing_table_name}")
conn.commit()
merged_df.to_sql(existing_table_name, conn, if_exists='replace', index=False)
conn.close()
print(merged_df.head())


             date_time  Super_Trend_signal  EMA_signal  Hybrid_signal  \
0  2020-01-01 00:00:00                  -1          -1             -1   
1  2020-01-01 04:00:00                  -1          -1             -1   
2  2020-01-01 08:00:00                   1          -1             -1   
3  2020-01-01 12:00:00                  -1           1             -1   
4  2020-01-01 16:00:00                  -1          -1             -1   

   Stochastic_signal  BB_signal  LSTM_Signals  
0                 -1         -1             1  
1                 -1         -1            -1  
2                 -1         -1             1  
3                 -1         -1            -1  
4                  1         -1            -1  


# Appending the GRU result in DB

In [9]:
import pandas as pd
import sqlite3
gru_df = pd.read_csv('gru_signals.csv')
conn = sqlite3.connect('indicators.db')
existing_table_name = 'indicators'
existing_df = pd.read_sql(f'SELECT * FROM {existing_table_name}', conn)
if len(gru_df) > len(existing_df):
    gru_df = gru_df.head(len(existing_df))
elif len(gru_df) < len(existing_df):
    padding_length = len(existing_df) - len(gru_df)
    padding_df = pd.DataFrame({'GRU_Signals': [0] * padding_length})
    gru_df = pd.concat([gru_df, padding_df], ignore_index=True)
existing_df['index'] = existing_df.index
gru_df['index'] = gru_df.index
merged_df = pd.merge(existing_df, gru_df[['index', 'GRU_Signals']], on='index', how='left')
merged_df.drop(columns='index', inplace=True)
merged_df.to_sql(existing_table_name, conn, if_exists='replace', index=False)
conn.close()
print(merged_df.head())


             date_time  Super_Trend_signal  EMA_signal  Hybrid_signal  \
0  2020-01-01 00:00:00                  -1          -1             -1   
1  2020-01-01 04:00:00                  -1          -1             -1   
2  2020-01-01 08:00:00                   1          -1             -1   
3  2020-01-01 12:00:00                  -1           1             -1   
4  2020-01-01 16:00:00                  -1          -1             -1   

   Stochastic_signal  BB_signal  LSTM_Signals  GRU_Signals  
0                 -1         -1             1           -1  
1                 -1         -1            -1           -1  
2                 -1         -1             1           -1  
3                 -1         -1            -1           -1  
4                  1         -1            -1           -1  


# Appending the Fb Prophet result in DB